<a href="https://colab.research.google.com/github/mmarze/ds-notebooks/blob/main/session_3/mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Zakres dzisiejszych zajęć:
* wdrożenie modelu ML

## Zarządzanie i wdrażanie modeli ML
Platform ML Flow (https://www.mlflow.org/docs/latest/index.html) umożliwia całościowe zarządzanie cyklem życia modeli.

Cykl życia modelu:

1. Pobranie surowych danych  2. Przygotowanie danych 3. Trenowanie 4. Wdrożenie

Inzynieria danych (1,2), Analityka danych (3), Inzynieria oprogramowania (4)

ML FLow ma 4 podstawowe komponenty:  TRACKING, PROJECTS, MODELS, REJESTR
* logowanie eksperymentów, wartości parametrów modeli i osiąganych przez nie wyników
* serializowanie modeli (na potrzeby współdzielenia modelu, przeniesienia na inne środowisko lub serwowania)
* samodzielny format do uruchamiania analiz
* wersjonowanie modelu, adnotowanie i przechowywanie w Rejestrze
Rozwiazanie open-source.


![mlflow](https://github.com/biodatageeks/ds-notebooks/blob/main/session_3/mlflow.jpeg?raw=1)

### Przygotowanie danych do analizy

In [1]:
user_name = 'mmarze' # TODO set your GitHub user name
tracking_uri = 'https://mlflow-server-919598915406.us-central1.run.app' # TODO set your mlflow server url
semester = '2024l' # TODO set appropriate semester
user_id = 7067 # TODO set appropriate user id

from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [2]:
!gsutil cp gs://ds-2024l-9102-notebook-data/survey/2020/survey_results_public.csv /content/

Copying gs://ds-2024l-9102-notebook-data/survey/2020/survey_results_public.csv...
\ [1 files][ 90.2 MiB/ 90.2 MiB]                                                
Operation completed over 1 objects/90.2 MiB.                                     


In [5]:
gs_path = f'/content/survey_results_public.csv'
table_name = "survey_2020"
spark.sql(f'DROP TABLE IF EXISTS {table_name}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true, NULLVALUE "NA") \
          LOCATION "{gs_path}"')

spark_df= spark.sql(f'SELECT *, CAST((convertedComp > 60000) AS STRING) AS compAboveAvg \
                    FROM {table_name} WHERE convertedComp IS NOT NULL ')

### Transformacja danych do wektora cech

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
y = 'compAboveAvg'
feature_columns = ['OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode']

stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c).setHandleInvalid("keep") for c in feature_columns]
stringindexer_stages += [StringIndexer(inputCol=y, outputCol='label')]

onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in feature_columns]
extracted_columns = ['onehot_' + c for c in feature_columns]
vectorassembler_stage = VectorAssembler(inputCols=extracted_columns, outputCol='features')

final_columns = [y] + feature_columns + extracted_columns + ['features', 'label']

transformed_df = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage]).fit(spark_df).transform(spark_df).select(final_columns)
training, test = transformed_df.randomSplit([0.8, 0.2], seed=1234) # Podzial na zbior treningowy/testowy

###  ML Flow: Definicja eksperymentu


In [8]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.1 MB/s eta 0:00:00


In [9]:
import mlflow
import mlflow.spark

mlflow.set_tracking_uri(tracking_uri)

ename = f"classifier_{user_name}"
mlflow.set_experiment(experiment_name=ename)
experiment = mlflow.get_experiment_by_name(ename)

2025/01/19 08:47:27 INFO mlflow.tracking.fluent: Experiment with name 'classifier_mmarze' does not exist. Creating a new experiment.


### Definicja metryk

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
evaluator_auroc = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")
evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_prec = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_f = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedFMeasure")

### ML Flow: logowanie eksperymentu z drzewem decyzyjnym

Gotowy kod potoku analizy danych instrumentalizujemy z wykorzystaniem ML Flow.
Instrumentalizacja nie wplywa na proces trenowania

In [12]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')
dt_model = Pipeline(stages=[dt]).fit(training)
pred_dt = dt_model.transform(test)
label_and_pred = pred_dt.select('label', 'prediction')

### Instrumentalizacja kodu z uzyciem Pythonowego API ML Flow

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="dt_model"):

    mlflow.set_tag("classifier", "decision_tree")  ## ustawienie tagow
    mlflow.log_param("depth", dt.getMaxDepth())    ## zapisanie metadanych - hiperparametrow

    res = dt_model.transform(test)

    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc)  ### zapisanie metryk
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc)
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall)
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f)

    # mlflow.spark.log_model(spark_model=dt_model, artifact_path='dt_classifier') ## logowanie artefaktu - serializowany model

🏃 View run dt_model at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9/runs/c1d71eec6c1b428b8d0bc97a4c63c114
🧪 View experiment at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9


ML Flow moze zapisac model z innej zaintegrowanej biblioteki (TF, Keras etc)
Co widzimy w ML Flow UI?

### ML Flow: logowanie eksperymentu drzewa decyzyjnego z walidacją krzyżową

In [13]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder(). \
    addGrid(dt.maxDepth, [2,3,4,5,6]).\
    build()
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator_auroc, numFolds=4)

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="best_model"):
    cv_model = cv.fit(training)

    mlflow.log_param("depth", cv_model.bestModel.depth)

    res = cv_model.bestModel.transform(test)

    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc)
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc)
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall)
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f)

    # mlflow.spark.log_model(spark_model=cv_model.bestModel, artifact_path='best_classifier')


🏃 View run best_model at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9/runs/31371c4f286d4343a589793990df18aa
🧪 View experiment at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9


### ML Flow: logowanie eksperymentu z modelem GBT
Wykorzystanie innego modelu, drzew decyzyjnych ze wzmocnieniem gradientowym.

In [14]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
gbt_model = gbt.fit(training)

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="gbt_model"):

    mlflow.log_param("depth", gbt.getMaxDepth())

    res = gbt_model.transform(test)

    test_metric_auroc = evaluator_auroc.evaluate(res)
    test_metric_acc = evaluator_acc.evaluate(res)
    test_metric_recall = evaluator_recall.evaluate(res)
    test_metric_prec = evaluator_prec.evaluate(res)
    test_metric_f = evaluator_f.evaluate(res)

    mlflow.log_metric(evaluator_auroc.getMetricName(), test_metric_auroc)
    mlflow.log_metric(evaluator_acc.getMetricName(), test_metric_acc)
    mlflow.log_metric(evaluator_recall.getMetricName(), test_metric_recall)
    mlflow.log_metric(evaluator_prec.getMetricName(), test_metric_prec)
    mlflow.log_metric(evaluator_f.getMetricName(), test_metric_f)

    # mlflow.spark.log_model(spark_model=gbt_model, artifact_path='gbt_classifier')

🏃 View run gbt_model at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9/runs/2b57ffcbb36b4e9a9ac566ee8510a007
🧪 View experiment at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9


### ML Flow: serwowanie modelu

Zeby zapewnić prosty interfejs do klasyfikatora, zapiszemy model wraz z krokami wstępnego przetwarzania (przekształcenie danych wejściowych w wektor cech)

Model jest przechowany w strukturze katalogowej - wraz z konfiguracją (zaleznosciami) oraz zserializowana reprezentacją. Moze byc przechowany w kilku "smakach" (flavour) w naszym przypadku jest to SparkML model oraz Pythonowa funkcja.

In [15]:
import mlflow
import mlflow.spark
from pyspark.ml.classification import GBTClassifier

raw_training, raw_test = spark_df.randomSplit([0.8, 0.2], seed=1234)

full_classifier_name=f"{user_name}_full_gbt_classifier"
version=1

with mlflow.start_run(experiment_id = experiment.experiment_id, run_name="gbt_model_raw"):
    pipeline = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage] + [gbt] )
    model = pipeline.fit(raw_training)
    # mlflow.spark.log_model(spark_model=model, artifact_path='gbt_classifier', registered_model_name=full_classifier_name)

🏃 View run gbt_model_raw at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9/runs/b49d549e04354a4890ebe03de61c88f8
🧪 View experiment at: https://mlflow-server-919598915406.us-central1.run.app/#/experiments/9


### Klasyfikacja jako serwis REST

Sekwencja komend do uruchomienia w terminalu

```bash
export USER_NAME=yourGitHubUserName # TODO change
export MLFLOW_TRACKING_URI=https://mlflow-server-919598915406.us-central1.run.app
export MODEL_VERSION=1
export MODEL_NAME=full_gbt_classifier
export MLFLOW_SERVE_PORT=9090
cd
/opt/conda/miniconda3/bin/python -m venv ./venv
source ./venv/bin/activate
mlflow models serve -m models:/${USER_NAME}_${MODEL_NAME}/${MODEL_VERSION} -p ${MLFLOW_SERVE_PORT} --no-conda
```

#### Przykładowe wywołanie serwisu

In [ ]:
import requests

url = "http://localhost:9090/invocations"
headers = {'Content-Type': 'application/json'}

input_data = {
    "dataframe_split": {
        "columns": ["OpSys", "EdLevel", "MainBranch", "Country", "JobSeek", "YearsCode"],
        "data": [
            [
                "MacOS",
                "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",
                "I am a developer by profession",
                "United Kingdom",
                "I am not interested in new job opportunities",
                "10"
            ]
        ]
    }
}

r = requests.post(url, json=input_data, headers=headers)
print(r.text)

In [ ]:
spark.stop()

# Podsumowanie kursu

Podczas zajęć przećwiczyliśmy następujące technologii i narzędzi z zakresu Data&AI:
Przechowywanie:
* rozproszony system plików (HDFS)
* obiektowy system plików (GCS)

Środowiska pracy:
* lokalny klaster Hadoop
* klaster Kubernetes w chmurze Google
* notatniki Jupyter

Przetwarzanie danych:
* Apache Spark, API DataFrame i SQL
* Pandas
* Python

Wizualizacja danych:
* matplotlib
* seaborn



![](https://github.com/biodatageeks/ds-notebooks/blob/main/img/ecosystem_green.png?raw=1)